In [ ]:
# Styling notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

In [4]:
class Scanner :
    def __init__(self, source, table) :
        self.source = source
        self.table = table

        # Initialize Scanner state
        self.position = 0           # First character
        self.currentText = None     # Current text/token
        self.currentToken = None    # Nothing beats good naming, right?
    
    def skip(self) :
        while self.source[self.position] == ' ':
            self.position += 1
            
    # Return class and text of the next token
    # 'eoI' is artificial and denotes "End of Input"
    # 'Unknown' is a class for text that doesn't translate into a proper token
    # Lots of assumptions in this, needs to be rewritten if
    #    a) we have numerical constants
    #    b) we have alphanumerical variable names
    #    c) it's Monday
    
    def nextToken(self) :
        if self.position >= len(self.source) :
            return 'eoI', 'eoI'                 # Done! Both are the same
        
        self.skip()
        text = ''                               # Accumulates the text
        c = self.source[self.position]
        if c.isdigit() :
            value = 0
            while c.isdigit() :
                value = value*10 + ord(c) - ord('0');
                self.position += 1
                c = self.source[self.position]
            self.currentText, self.currentToken = value, 'numConstant'
        elif not c.isalpha() : # that's for '(', ')' and operators and and and...
            text += c
            self.position += 1  # only one char
            self.currentText, self.currentToken = text, self.table.get(text,'unknown')
        else :
            # Collect alpha string, works for ops, vars and consts
            while c.isalpha() : 
                text += c
                self.position += 1
                c = self.source[self.position]
                self.currentText, self.currentToken = text, self.table.get(text,'identifier')
                
        # Got the token text, look up the class in the dictionary and call it quits
        return self.currentToken, self.currentText

In [5]:
# A. Token table (easy!)
table = {
    '(' : 'oParen',
    ')' : 'cParen',
    '+' : 'arithOp2',
    '-' : 'arithOp2',
    '*' : 'arithOp2',
    '/' : 'arithOp2',
}

In [6]:
s = Scanner('(a + 1) ',table)

print(s.nextToken())
print(s.nextToken())
print(s.nextToken())
print(s.nextToken())

('oParen', '(')
('identifier', 'a')
('arithOp2', '+')
('numConstant', 1)
